<img src="../../docs/images/DSPy8.png" alt="DSPy7 图片" height="150"/>

## **DSPy 断言**: 对基础模型施加计算约束

### **QuizGen**: 生成多项选择题问题

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/stanfordnlp/dspy/blob/main/examples/quiz/quiz_assertions.ipynb)


这个笔记本突出了[**DSPy断言**](https://dspy-docs.vercel.app/docs/building-blocks/assertions)的一个示例，允许在DSPy程序中声明计算约束。


这个笔记本建立在**DSPy**框架的基本概念之上。跟随这个笔记本的先决条件是已经阅读了[DSPy教程](../../intro.ipynb)，[**DSPy断言文档**](https://dspy-docs.vercel.app/docs/building-blocks/assertions)以及LongFormQA中的DSPy断言入门教程(../longformqa/longformqa_assertions.ipynb)。

In [ ]:
# 克隆代码库
!git clone https://huggingface.co/arnavs11/DSPy_QuizGen_Cache
%cd DSPy_QuizGen_Cache/
!git checkout master
%cd ..
import os
repo_clone_path = '/content/DSPy_QuizGen_Cache'

# 检查'/content'目录是否可写
if not os.access('/content', os.W_OK):
    # 如果'/content'目录不可写，选择另一个目录
    # 例如：使用相对于当前工作目录的目录
    repo_clone_path = os.path.join(os.getcwd(), 'DSPy_QuizGen_Cache')

# 为此笔记本设置缓存
os.environ["DSP_NOTEBOOK_CACHEDIR"] = repo_clone_path

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import regex as re
import json

try: # 当在谷歌Colab上时，让我们克隆笔记本以便下载缓存。
    import google.colab
    repo_path = 'dspy'
    
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)


import pkg_resources # 如果尚未安装该软件包，则安装该软件包
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    !pip install -e $repo_path

import dspy
from dspy.predict import Retry
from dspy.datasets import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.evaluate.evaluate import Evaluate
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

In [ ]:
# 创建一个 ColBERTv2 模型，连接到指定的 URL
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# 配置 DeepSpeed 设置，使用上面创建的 ColBERTv2 模型
dspy.settings.configure(rm=colbertv2_wiki17_abstracts)

# 创建一个 OpenAI 模型，使用指定的 GPT-3.5 Turbo 模型和最大 token 数为 500
turbo = dspy.OpenAI(model='gpt-3.5-turbo-0613', max_tokens=500)

# 配置 DeepSpeed 设置，使用上面创建的 OpenAI 模型，同时设置 trace 为空列表，温度为 0.7
dspy.settings.configure(lm=turbo, trace=[], temperature=0.7)

In [ ]:


# 创建HotPotQA数据集对象，指定训练集种子为1，训练集大小为300，评估集种子为2023，开发集大小为300，测试集大小为0，保留详细信息
dataset = HotPotQA(train_seed=1, train_size=300, eval_seed=2023, dev_size=300, test_size=0, keep_details=True)

# 从训练集中提取问题和答案，构建训练集
trainset = [x.with_inputs('question', 'answer') for x in dataset.train]

# 从开发集中提取问题和答案，构建开发集
devset = [x.with_inputs('question', 'answer') for x in dataset.dev]

### 3] QuizGen

让我们介绍一个新任务：QuizGen。

QuizGen接收HotPotQA数据点，并将其转换为带有相应选项的多项选择测验问题。每个问题的选项集以JSON键值对格式生成。在这种情况下，我们指定生成4个选项。

通过这个程序，我们的目标是生成符合以下准则的测验选项：
1. 生成的选项以JSON格式呈现。
2. 生成的选项包括正确答案。
3. 生成的选项除了正确答案外还包括可信的干扰选项。

In [ ]:
class GenerateAnswerChoices(dspy.Signature):
    """生成包含指定问题的正确答案和可信干扰项的JSON格式答案选项。"""
    question = dspy.InputField()
    correct_answer = dspy.InputField()
    number_of_choices = dspy.InputField()
    answer_choices = dspy.OutputField(desc='JSON键值对')

class QuizAnswerGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_choices = dspy.ChainOfThought(GenerateAnswerChoices)

    def forward(self, question, answer):
        choices = self.generate_choices(question=question, correct_answer=answer, number_of_choices=number_of_choices).answer_choices
        return dspy.Prediction(choices = choices)

number_of_choices = '4'
quiz_generator = QuizAnswerGenerator()

### 4] 评估 - 内在和外在

#### 内在度量：通过内部计算约束的目标

**有效格式** - 输出的答案选择应该是JSON格式，经过解析键值对后进行验证。

**正确答案包含** - 这是一个一般性检查，以确保生成的测验选择实际上包含问题的正确答案。

**合理的干扰项** - 这个验证是为了检查生成的选择是否包含干扰项答案选项，这些选项是问题的合理答案。我们定义并调用另一个**DSPy**程序：``Predict``在``AssessQuizChoices``上，依赖于相同的LM来回答问题：`“答案选择中的干扰项是否合理，且不容易识别为不正确？”`

In [ ]:

def format_checker(choice_string):
    # 检查输入的选项字符串是否符合要求
    try:
        choices = json.loads(choice_string)
        if isinstance(choices, dict) and all(isinstance(key, str) and isinstance(value, str) for key, value in choices.items()):
            return True
    except json.JSONDecodeError:
        return False

    return False

def is_correct_answer_included(correct_answer, generated_choices):
    # 检查正确答案是否包含在生成的选项中
    try:
        choices_dict = json.loads(generated_choices)
        return correct_answer in choices_dict.values()
    except json.JSONDecodeError:
        return False

def is_plausibility_yes(assessment_answer):
    """检查评估答案的第一个单词是否为'yes'。"""
    return assessment_answer.split()[0].lower() == 'yes'
    
class AssessQuizChoices(dspy.Signature):
    """沿着指定维度评估测验答案选项的质量。"""
    
    question = dspy.InputField()
    answer_choices = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="是或否")

def format_valid_metric(gold, pred, trace=None):
    generated_choices = pred.choices
    format_valid = format_checker(generated_choices)
    score = format_valid
    return score

def is_correct_metric(gold, pred, trace=None):
    correct_answer, generated_choices = gold.answer, pred.choices
    correct_included = is_correct_answer_included(correct_answer, generated_choices)
    score = correct_included
    return score

def plausibility_metric(gold, pred, trace=None):
    question, generated_choices = gold.question, pred.choices
    plausibility_question = "答案选项中的干扰项是否合理且不容易识别为不正确？"
    plausibility_assessment = dspy.Predict(AssessQuizChoices)(question=question, answer_choices=generated_choices, assessment_question=plausibility_question)
    plausibility_result = plausibility_assessment.assessment_answer.split()[0].lower() == 'yes'
    score = plausibility_result
    return score

#### 外部度量：评估生成的输出在下游任务中的整体质量和有效性

外部度量被定义为生成的测验选项的整体质量，并通过一个综合度量进行评估，考虑了这些约束条件。

综合度量保持了生成有效的测验选项所需的核心内在度量，验证有效的格式和正确答案的包含，并且整体综合度量返回了三个内在度量的平均分数。

In [ ]:
def overall_metric(gold, pred, trace=None):
    # 从gold中获取问题、正确答案和生成的选项
    question, correct_answer, generated_choices = gold.question, gold.answer, pred.choices
    # 检查生成的选项的格式是否有效
    format_valid = format_checker(generated_choices)
    # 检查正确答案是否包含在生成的选项中
    correct_included = is_correct_answer_included(correct_answer, generated_choices)
    # 判断问题：生成的干扰项是否合理且不易识别为不正确？
    plausibility_question = "Are the distractors in the answer choices plausible and not easily identifiable as incorrect?"
    # 进行可信度评估
    plausibility_assessment = dspy.Predict(AssessQuizChoices)(question=question, answer_choices=generated_choices, assessment_question=plausibility_question)
    # 判断可信度评估结果是否为"yes"
    plausibility_result = plausibility_assessment.assessment_answer.split()[0].lower() == 'yes'
    # 计算得分，如果正确答案包含在生成的选项中且格式有效，则计算得分，否则得分为0
    score = (format_valid + correct_included + plausibility_result) / 3.0 if correct_included and format_valid else 0
    return score

因此，我们将评估定义如下：

In [ ]:
# 定义评估指标列表
metrics = [format_valid_metric, is_correct_metric, plausibility_metric, overall_metric]

# 遍历评估指标列表
for metric in metrics:
    # 创建 Evaluate 实例，传入评估指标、开发集、线程数、显示进度和显示表格参数
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 使用评估器对 quiz_generator 进行评估
    evaluate(quiz_generator)

让我们来看一个示例测验选项生成：

In [ ]:
# 选择第67个样本作为示例
example = devset[67]
# 使用问题生成器生成问题选项
quiz_choices = quiz_generator(question=example.question, answer=example.answer)
# 打印生成的问题选项
print(f'生成的问题选项: ', quiz_choices.choices)

In [ ]:
# 对于每一个指标，创建一个Evaluate对象，并传入相应参数
for metric in metrics:
    evaluate = Evaluate(metric=metric, devset=devset[67:68], num_threads=1, display_progress=True, display_table=5)
    # 调用evaluate对象，传入quiz_generator参数
    evaluate(quiz_generator)

我们看到生成的测验选项没有保持有效的 JSON 格式，这违反了有效格式和正确性检查，尽管这些选项被认为是合理的。我们还看到正确答案也被标记为“(正确答案)”，这并不是产生良好测验问题答案选择的意图。

让我们看看如何集成 DSPy 断言并施加约束以产生更好的答案选择。

### 5] 引入断言：使用断言生成答题器

让我们在 DSPy 断言语义中包含简单重申我们的计算约束的断言。

在第一个**断言**中，我们检查生成的测验选择是否为 JSON 格式，如果不是，则断言：**"答案选择的格式应为 JSON 格式。请相应进行修订。"**

我们还检查测验选择集合是否包含正确答案，并在违反时确保这一点，并提供反馈信息：**"答案选择不包括问题的正确答案。请相应进行修订。"**

最后，我们评估可信的干扰选择是否确实是良好的干扰选项，如果不是，则断言：**"答案选择不是可信的干扰选项，或者太容易被识别为不正确。请进行修订，提供更具挑战性和可信度的干扰选项。"**

In [ ]:
class QuizAnswerGeneratorWithAssertions(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_choices = dspy.ChainOfThought(GenerateAnswerChoices)

    def forward(self, question, answer):
        # 生成备选答案选项字符串
        choice_string = self.generate_choices(question=question, correct_answer=answer, number_of_choices=number_of_choices).answer_choices
        # 检查答案选项的格式是否为JSON格式
        dspy.Suggest(format_checker(choice_string), "答案选项的格式应为JSON格式。请相应修改。", target_module=GenerateAnswerChoices)
        # 检查答案选项是否包含正确答案
        dspy.Suggest(is_correct_answer_included(answer, choice_string), "答案选项不包含问题的正确答案。请相应修改。", target_module=GenerateAnswerChoices)
        # 判断是否干扰项在答案选项中是否合理且不易被识别为错误
        plausibility_question = "干扰项在答案选项中是否合理且不易被识别为错误？"
        plausibility_assessment = dspy.Predict(AssessQuizChoices)(question=question, answer_choices=choice_string, assessment_question=plausibility_question)
        # 判断干扰项是否合理
        dspy.Suggest(is_plausibility_yes(plausibility_assessment.assessment_answer), "答案选项中的干扰项不合理或太容易被识别为错误。请修改以提供更具挑战性和合理性的干扰项。", target_module=GenerateAnswerChoices)
        return dspy.Prediction(choices = choice_string)

number_of_choices = '4'
# 断言转换模块
quiz_generator_with_assertions = assert_transform_module(QuizAnswerGeneratorWithAssertions().map_named_predictors(Retry), backtrack_handler)

让我们现在在开发集上评估`QuizAnswerGeneratorWithAssertions`。

In [ ]:
# 定义评估指标列表
metrics = [format_valid_metric, is_correct_metric, plausibility_metric, overall_metric]

# 遍历评估指标列表
for metric in metrics:
    # 创建 Evaluate 对象，传入指标、开发集、线程数、显示进度和显示表格参数
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 使用评估对象对 quiz_generator_with_assertions 进行评估
    evaluate(quiz_generator_with_assertions)

现在让我们看看随着断言的添加，我们生成的测验选项集是如何改进的。

In [ ]:
# 选择第67个样本作为示例
example = devset[67]
# 使用带有断言的quiz_generator生成问题和答案选项
quiz_choices = quiz_generator_with_assertions(question=example.question, answer=example.answer)
# 打印生成的测验选项
print(f'生成的测验选项: ', quiz_choices.choices)

In [ ]:
# 对于每个指标，创建一个Evaluate对象并进行评估
for metric in metrics:
    evaluate = Evaluate(metric=metric, devset=devset[67:68], num_threads=1, display_progress=True, display_table=30)
    # 使用quiz_generator_with_assertions对evaluate对象进行评估
    evaluate(quiz_generator_with_assertions)

我们看到，测验选项符合我们所有的约束条件！

不仅所有答案选项都是合理的，并且已经移除了任何指示正确答案的标志，而且答案选项现在保持了有效的JSON格式，提供了4个可能的答案选项，其中包括正确答案。

### 6] 使用断言进行编译

我们可以利用 **DSPy** 的 `BootstrapFewShotWithRandomSearch` 优化器，自动生成少样本演示，并对候选进行随机搜索，以输出最佳编译程序。我们通过 `final_metric` 综合指标来评估这一过程。

我们可以首先在 `QuizAnswerGenerator` 上进行评估，看看在不包含断言的情况下编译的表现如何。

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，指定评估指标为overall_metric，最大bootstrap演示次数为2，候选程序数量为6
teleprompter = BootstrapFewShotWithRandomSearch(metric=overall_metric, max_bootstrapped_demos=2, num_candidate_programs=6)
# 编译quiz_generator生成的学生和老师模型，使用trainset进行训练，使用devset的前100个样本进行验证
compiled_quiz_generator = teleprompter.compile(student=quiz_generator, teacher=quiz_generator, trainset=trainset, valset=devset[:100])

# 遍历metrics列表中的每个评估指标
for metric in metrics:
    # 创建一个Evaluate对象，指定评估指标为当前metric，使用devset进行评估，线程数为1，显示进度，每次显示前5个结果
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对编译后的quiz_generator模型进行评估
    evaluate(compiled_quiz_generator)

现在我们在两种设置下测试编译与断言：

**带断言的编译**：在编译过程中进行基于断言的示例引导和反例引导。教师拥有断言，而学生没有，因为学生从教师的基于断言的引导示例中学习。

**带断言的编译+推理**：为教师和学生提供基于断言的优化，以在编译和推理过程中提供增强的基于断言的输出。

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，指定评估指标为overall_metric，最大bootstrap演示次数为2，候选程序数量为6
teleprompter = BootstrapFewShotWithRandomSearch(metric=overall_metric, max_bootstrapped_demos=2, num_candidate_programs=6)
# 编译带有断言的测验生成器，使用teleprompter对象进行编译，指定学生为quiz_generator，老师为quiz_generator_with_assertions，训练集为trainset，验证集为devset的前100个样本
compiled_with_assertions_quiz_generator = teleprompter.compile(student=quiz_generator, teacher=quiz_generator_with_assertions, trainset=trainset, valset=devset[:100])

# 遍历metrics列表中的每个评估指标
for metric in metrics:
    # 创建一个Evaluate对象，指定评估指标为当前metric，验证集为devset，线程数为1，显示进度为True，显示表格为5
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对编译后的带有断言的测验生成器进行评估
    evaluate(compiled_with_assertions_quiz_generator)

In [ ]:
# 创建一个BootstrapFewShotWithRandomSearch对象，使用overall_metric作为度量标准，最大引导演示次数为2，候选程序数量为6
teleprompter = BootstrapFewShotWithRandomSearch(metric=overall_metric, max_bootstrapped_demos=2, num_candidate_programs=6)
# 编译带有断言的测验生成器，使用teleprompter作为引导器，学生和老师都是quiz_generator_with_assertions，训练集为trainset，验证集为devset的前100个样本
compiled_quiz_generator_with_assertions = teleprompter.compile(student=quiz_generator_with_assertions, teacher=quiz_generator_with_assertions, trainset=trainset, valset=devset[:100])

# 遍历metrics列表
for metric in metrics:
    # 创建一个Evaluate对象，使用指定的度量标准metric，验证集为devset，线程数为1，显示进度条，每5个显示一次表格
    evaluate = Evaluate(metric=metric, devset=devset, num_threads=1, display_progress=True, display_table=5)
    # 对编译后的带有断言的测验生成器进行评估
    evaluate(compiled_quiz_generator_with_assertions)